# Оптимизация алгоритмов

Задача. Чтобы различать короткие РНК по составу нуклеотидов для начала надо найти какой состав вообще возможен в короткой последовательности, например для пептидов из 5 аминокислот.

В результате мы должны иметь массив из 4-хчисельных комбинаций, которые в сумме дают 15.

Для сравнения производительности все решения будем оформлять в виде функций.

In [9]:
N = 15

Самый простой способо - перебора (bruteforce)

In [28]:
def solution_brute():
    ACGU = []  # сюда будет складывать комбинации типа (3,3,6,0)
    for A in range(N+1): # +1 т.к. может быть (0,0,0,12)
        for C in range(N+1):
            for G in range(N+1):
                for U in range(N+1):
                    if A+C+G+U == N:
                        ACGU.append((A,C,G,U))
    return ACGU

len(solution_brute())

816

Для каждой возможной комбинации мы выполняем отдельную итерацию и проверку на совпадение суммы.

Для типичных задач с перебором многих комбинаций можно использовать подходящие методы из библиотеки `itertools`. Объем вычислений будет тот же, но запись компактнее.

In [29]:
import itertools

def solution_itertools():
    return [(A,C,G,U) for (A,C,G,U) in itertools.product(range(N+1),repeat=4) if A+C+G+U == N]

len(solution_itertools())

816

Умное решение позволяет перебирать не все комбинации с последующей проверкой, а только те, которые сразу дают решения.

In [30]:
def solution_clever():
    return [(A,C,G, (N - A - C - G))
            for A in range(N+1)
            for C in range(N+1 - A)
            for G in range(N+1 - A - C)]

len(solution_clever())

816

В `numpy` задачу можно представить в 4-х мерном виде. Но так как четвертое число определяется тремя другими, то достаточно 3-мерного.

Все комбинации из 3 индексов мы выстаиваем в ряд и высчитываем отклонение их суммы от искомого.

На последнем шаге отбрасываем все комбинации, где отклонение отрицательное.

In [31]:
import numpy as np

In [32]:
def solution_np():
    ACG = np.indices((N+1,N+1,N+1)).reshape(3,(N+1)**3)
    U = N - ACG.sum(axis=0)
    return np.vstack((ACG, U)).T[U > -1]
len(solution_np())

816

Модифицируем немного, чтобы отбрасывать лишние комбинации на промежуточном шаге.

In [37]:
def solution_np_econom():
    ACG = np.indices((N+1,N+1,N+1)).reshape(3,(N+1)**3)
    U = N - ACG.sum(axis=0)
    bb = U > -1
    return np.vstack((ACG[:,bb], U[bb])).T
len(solution_np_econom())

816

Давайте замерим производительность решений.

In [56]:
%timeit solution_brute()  # 10 ms
%timeit solution_itertools()  # 9.5 ms
%timeit solution_clever()  # 0.2 ms
%timeit solution_np()  # 0.1 ms
%timeit solution_np_econom()  # 0.11 ms

9.96 ms ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.44 ms ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
194 µs ± 43 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
107 µs ± 59.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
109 µs ± 62.7 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Первые три решения возвращают список, а другие два - `ndarray`

In [47]:
print(type(solution_clever()))
print(type(solution_np()))

<class 'list'>
<class 'numpy.ndarray'>


Заменим квадратные скобки на обычные, так чтобы функция возвращала генератор.

In [54]:
def solution_clever_generator():
    return ((A,C,G, (N - A - C - G))
            for A in range(N+1)
            for C in range(N+1 - A)
            for G in range(N+1 - A - C))

print(type(solution_clever_generator()))

<class 'generator'>


In [55]:
%timeit solution_clever_generator()  # 0.001 ms

1.02 µs ± 3.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Ускорение в тысячу раз связано с отсутствием выполнения каких-либо операций: генератор приступает к работе только при запросе результата.  

Если мы соберем все результаты в список (с выделением памяти), то получим те же 0.2 мс.

In [51]:
%timeit list(solution_clever_generator())  # 0.23 ms

230 µs ± 56.8 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Если нам не нужно хранить все результаты в одной переменной, то мы можем сэкономить на памяти, незначительно потеряв на производительности.

In [61]:
def count_generator():
    n=0
    for _ in solution_clever_generator():
        n+=1
    return n

%timeit count_generator()  # 0.27 ms

272 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Выводы

1. `numpy` гораздо эффективнее обращается с большими массивами памяти. Это дает существенное ускорение. 
2. Оптимизация в `numpy` часто мнимая - попытка сократить уже созданный массив за счет создания еще одного приводит к замедлению, ибо операции с выделением памяти самые затратные.
3. Если возможно решение, когда можно работать с каждым объектом независимо (в памяти не нужно держать весь массив), то такое решение и быстрое и масштабируемое. 